<a href="https://colab.research.google.com/github/Buitragox/Crowdsourcing-Thesis/blob/main/notebooks/FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import skimage.io as io
import zipfile

import tensorflow as tf
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential

**Download utils.py**

In [ ]:
#Get utils.py
!wget --no-check-certificate 'https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/utils.py' -O utils.py

--2024-05-26 19:07:11--  https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5405 (5.3K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   5.28K  --.-KB/s    in 0s      

2024-05-26 19:07:11 (79.3 MB/s) - ‘utils.py’ saved [5405/5405]



In [ ]:
from utils import load_ids, load_pickle_ma

**Download pickle file**

In [ ]:
!wget --no-check-certificate 'https://github.com/Buitragox/Crowdsourcing-Thesis/raw/main/data/pkl/train_crowdsourced_labels.pkl' -O train_crowdsourced_labels.pkl

--2024-01-07 21:16:28--  https://github.com/Buitragox/Crowdsourcing-Thesis/raw/main/BCSS/pkl/train_crowdsourced_labels.pkl
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/pkl/train_crowdsourced_labels.pkl [following]
--2024-01-07 21:16:29--  https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/pkl/train_crowdsourced_labels.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10426477 (9.9M) [application/octet-stream]
Saving to: ‘train_crowdsourced_labels.pkl’

train_crowdsourced_ 100%[===================>]   9.94M  63.2MB/s    in 

**Download dataset images from Drive**

In [ ]:
!pip install gdown

In [ ]:
#Train
!gdown 1c2mrPeND5-jxL43Im8eDi5PCyuTGBLEX -O train.zip
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1c2mrPeND5-jxL43Im8eDi5PCyuTGBLEX' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1c2mrPeND5-jxL43Im8eDi5PCyuTGBLEX" -O train.zip && rm -rf /tmp/cookies.txt

In [ ]:
#Test
!gdown 1n_zGW0qht4rQauar3hQkqpz8sreb0SRv -O test.zip
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1n_zGW0qht4rQauar3hQkqpz8sreb0SRv' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1n_zGW0qht4rQauar3hQkqpz8sreb0SRv" -O test.zip && rm -rf /tmp/cookies.txt

**Unzip**

In [ ]:
with zipfile.ZipFile("./train.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [ ]:
with zipfile.ZipFile("./test.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

**Load pickle file**

In [ ]:
pkl_path = "./train_crowdsourced_labels.pkl"
images_path = "./Train_non_experts_simple/"
test_path = "./Test/"
batch_size = 8
R = 20

In [ ]:
ids, labels = load_pickle_ma(pkl_path, images_path, R)

### Feature Extraction

In [ ]:
from keras.applications.vgg16 import VGG16

# Load VGG16 model without the top layer (fully connected layers)
model = Sequential()
model.add(VGG16(include_top=False))
# Add pooling layer to reduce number of features to a 512 vector.
model.add(tf.keras.layers.AveragePooling2D(pool_size=(7, 7), strides=(7, 7), padding="same"))
model.add(Flatten())
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 average_pooling2d (Average  (None, None, None, 512)   0         
 Pooling2D)                                                      
                                                                 
 flatten (Flatten)           (None, None)              0         
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 14714688 (56.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
class DataSeq2(tf.keras.utils.Sequence):

    def __init__(self, ids, labels, batch_size=8, image_size=224, norm=False):
        self.ids = ids
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.norm = norm

    def __load__(self, id_name):
        image_path = id_name
        image = io.imread(image_path)
        if self.norm:
            image = image / 255.0

        return image

    def __getitem__(self, index):
        n = self.batch_size

        if (index + 1) * n > len(self.ids):
            n = len(self.ids) - index * self.batch_size

        files_batch = self.ids[index * n : (index + 1) * n]
        labels_batch = self.labels[index * n : (index + 1) * n]
        images = []

        for id in files_batch:
            image = self.__load__(id)
            images.append(image)

        images = np.array(images)
        labels_batch = np.array(labels_batch)

        return images, labels_batch

    def on_epoch_end(self):
        pass

    def __len__(self):
        return int(np.ceil(len(self.ids) / self.batch_size))

In [ ]:
# Extract features using a generator
def extract_features_dataseq(images_seq, model):
    features = model.predict(images_seq)
    return features

In [ ]:
normalize = False
images_seq = DataSeq2(ids, labels, norm=normalize)
img = images_seq[0][0]
print(img.shape)
print(img.min(), img.max())

(8, 224, 224, 3)
13 255


In [ ]:
features = extract_features_dataseq(images_seq, model)
features.shape

9406/9406 [==============================] - 437s 46ms/step


(75243, 512)

In [ ]:
train_data = np.array(features)
print(train_data.shape)
print(train_data)

(75243, 512)
[[0.85887575 2.0172694  5.434412   ... 0.5864428  0.93910575 3.4984381 ]
 [1.1397283  1.0173965  2.5118337  ... 0.         0.8767194  0.1290536 ]
 [2.596618   0.8754663  8.503624   ... 0.13184737 0.17404003 0.9714356 ]
 ...
 [1.8762636  0.         1.3436652  ... 0.         0.45615667 0.        ]
 [0.2417545  0.         3.454934   ... 0.02765862 5.4303455  0.        ]
 [1.0983104  0.         3.8425934  ... 0.         1.4552803  0.08967637]]


In [ ]:
np.save('TrainData.npy', train_data)

In [ ]:
test_ids, test_labels = load_ids(test_path)
test_seq = DataSeq2(test_ids, test_labels, norm=normalize)

In [ ]:
test_features = extract_features_dataseq(test_seq, model)
test_features.shape

546/546 [==============================] - 29s 47ms/step


(4364, 512)

In [ ]:
test_data = np.array(test_features)
print(test_data.shape)
print(test_data)

(4364, 512)
[[ 0.          0.          2.697621   ...  0.          3.0490682
   0.35847804]
 [ 0.          0.          4.763428   ...  0.5938789   1.5368123
   0.        ]
 [ 0.          0.          3.9389405  ...  0.          1.9051918
   0.        ]
 ...
 [ 0.86279356  0.          7.625779   ...  0.          2.4714165
   0.        ]
 [ 0.          0.          4.093834   ...  0.          1.0540651
   0.02058   ]
 [ 0.56125194  0.         13.438762   ...  0.          1.6440294
   0.        ]]


In [ ]:
np.save('TestData.npy', test_data)

In [ ]:
np.save('TestLabels.npy', test_labels)